In [3]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from itertools import permutations

files = [f for f in listdir("C:/qarch/qsurface/data/monolithic") if isfile(join("C:/qarch/qsurface/data/monolithic", f))]
new_location = ["C:/qarch/qsurface/data/monolithic/normalized/"+ f for f in files]
FILES = ["C:/qarch/qsurface/data/monolithic/"+ f for f in files]

In [4]:
''' This block creates a separate dataframe and fills it up with the new combination of error string and uses
the rest same values. And merges them later with a resetted index.'''

for new_loc, location in zip(new_location, FILES):
    data = pd.read_csv(location, sep=';')

    data2 = pd.DataFrame(columns=data.columns)
    for index, row in data.iterrows():
        perms = [''.join(p) for p in set(permutations(row["error_config"]))]
        for permutation in perms:
            if permutation == row["error_config"]:
                continue
            else:
                data2.loc[data2.shape[0]] = [permutation, row['lie'], row['p'], row['s'],0,0,0,0,0,0,0,0,0,0,0,0,0]

    data_merged = pd.concat([data, data2]).reset_index()
    del data_merged['index']

    '''Now we need to rescale the error permutation star and plaquette weights. And later normalize the whole superoperator.'''
    # Finding the right weights

    for index, row in data.iterrows():
        perms = [''.join(p) for p in set(permutations(row["error_config"]))]
        n_perms = len(perms)

        for index_f, row_f in data_merged.iterrows():
            if row_f['error_config'] in perms and row_f['lie'] == row['lie']:
                data_merged.loc[index_f,'p'] = row_f['p']/n_perms
                data_merged.loc[index_f,'s'] = row_f['s']/n_perms

    # Normalizing the whole CSV weights

    psum = data_merged['p'].sum()
    ssum = data_merged['s'].sum()

    data_merged['p'] = data_merged['p'].div(psum)
    data_merged['s'] = data_merged['s'].div(ssum)

    data_merged.to_csv(new_loc, sep=';', index=False)